In [1]:
import pandas as pd
from tqdm import tqdm
import psycopg2
import time 

import pandas as pd
import os
import numpy as np

from sentence_transformers import SentenceTransformer, LoggingHandler
import logging
import tqdm as notebook_tqdm


/home/fs01/spec1142/anaconda3/envs/patents/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [1]:
f = open('/home/fs01/spec1142/Emma/GateKeepers/' + "database.txt", "r")
user , password = f.read().split()

main_path = '/home/fs01/spec1142/Emma/GateKeepers/Text_encoding/'


## Get papers that are not encoded yet

In [ ]:
##query the work_ids that are in the works table but not in the encoded_works table


start = time.time()
#establishing the connection
conn = psycopg2.connect("user=" + user + " password=" + password)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()


#Creating table as per requirement
sql ="""SELECT work_id 
        FROM ( SELECT work_id , title , abstract FROM works_OpenAlex WHERE abstract != '' AND  abstract != 'nan') AS w
        WHERE  NOT EXISTS (
                   SELECT work_id
                   FROM   encoded_works_OpenAlex
                   WHERE  work_id = w.work_id
                   )
        ;"""
cursor.execute(sql)
result = cursor.fetchall()

end = time.time()
print(end - start)

#Closing the connection
conn.close()





dic_works = {} 

for line in tqdm(result): 
    work_id = line[0]
    dic_works[work_id] = {}
    
    

## save the ids
    
import json
json = json.dumps(dic_works)
f = open(path + 'new_papers_to_encode.json',"w")
f.write(json)
f.close()



## Get text (title and abstract) lf the papers that are not encoded yet

run "get_texts.py"

## Encode the texts 

run "encoding.py"                   

## Load encoding to the database

### Create table (if does not already exist)

In [ ]:
table_name = "encoded_works_OpenAlex"

schema ="""CREATE TABLE  """ + table_name + """ (
   work_id VARCHAR(20),
   encoded_title TEXT,
   encoded_abstract TEXT
   );"""



#establishing the connection
conn = psycopg2.connect("user=" + user + " password=" + password)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Doping EMPLOYEE table if already exists.
cursor.execute("DROP TABLE IF EXISTS " + table_name)

#Creating table as per requirement

sql = schema


cursor.execute(sql)
print("Table created successfully........")
conn.commit()
##Closing the connection
conn.close()

### Load data

In [1]:
index_columns = ['work_id']

table_name = "encoded_works_OpenAlex"

#establishing the connection
conn = psycopg2.connect("user=" + user + " password=" + password)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

for k in tqdm(range(13)):
    
    
    file = "new_works_encoded_" + str(k) + ".tsv"
    path =  main_path + "OA_encoded_works/" + file

    with open(path) as f:
        cursor.copy_expert("COPY " + table_name + " FROM STDIN WITH DELIMITER E'\t' CSV HEADER", f)

    conn.commit()
    print(k)



#Closing the connection
conn.close()




/home/fs01/spec1142/anaconda3/envs/patents/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
  9%|▉         | 1/11 [03:53<38:51, 233.14s/it]

2


 18%|█▊        | 2/11 [07:14<32:08, 214.27s/it]

3


 27%|██▋       | 3/11 [10:15<26:35, 199.41s/it]

...

10


 91%|█████████ | 10/11 [33:31<02:58, 178.23s/it]

11


100%|██████████| 11/11 [36:05<00:00, 196.91s/it]

12


### Index table 

In [ ]:
##establishing the connection
conn = psycopg2.connect("user=" + user + " password=" + password)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
for index_column in index_columns:
   #Creating table as per requirement
    sql ='''CREATE INDEX ''' + index_column + '_' + table_name + ''' ON '''+ table_name +'''(''' + index_column + ''');'''
    cursor.execute(sql)
    print("Table indexed successfully........")
conn.commit()
#Closing the connection
conn.close()


## Load data to GoogleCloud

### upload data from Google Cloud to the database


In [ ]:
from google.cloud import storage
from tqdm import tqdm
import psycopg2

## connection to Google Cloud 
path_to_private_key = main_path + "openalex-lee-c532eb059285.json"
client_storage = storage.Client.from_service_account_json(json_credentials_path=path_to_private_key)
bucket = client_storage.bucket('openalex-lee')

# Get blobs in specific subirectory
blobs_specific = list(bucket.list_blobs(prefix='OpenAlex/works_encoded/'))
list_files = [ str(elem).split(", ")[1] for elem in blobs_specific][::-1]


for file in tqdm(list_files[13+14+3:]):
    blob = bucket.blob(file)

    ##establishing the connection
    conn = psycopg2.connect("user=" + user + " password=" + password)

    ##Creating a cursor object using the cursor() method
    cursor = conn.cursor()

    table_name = "encoded_works_OpenAlex"

    with blob.open("r") as f:
        cursor.copy_expert("COPY " + table_name + " FROM STDIN WITH DELIMITER E'\t' CSV HEADER WHERE LEFT(work_id,1) = 'W'", f)

    conn.commit()

    #Closing the connection
    conn.close()
    
    

### Upload file from local to Google Cloud

In [1]:
from google.cloud import storage
from tqdm import tqdm

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    
    path_to_private_key = "/home/fs01/spec1142/Emma/openalex-lee-c532eb059285.json"
    client_storage = storage.Client.from_service_account_json(json_credentials_path=path_to_private_key)
    bucket = client_storage.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )
    
    
for k in tqdm(range(13)):
    bucket_name = 'openalex-lee'
    destination_blob_name = "OpenAlex/works_encoded/" + "new_works_encoded_" + str(k) + ".tsv"
    source_file_name = main_path + "new_works_encoded_" + str(k) + ".tsv"
    upload_blob(bucket_name, source_file_name, destination_blob_name)



  9%|▉         | 1/11 [02:49<28:17, 169.73s/it]

File /home/fs01/spec1142/Emma/Download_OpenAlex/OA_encoded_works/new_works_encoded_2.tsv uploaded to OpenAlex/works_encoded/new_works_encoded_2.tsv.


 18%|█▊        | 2/11 [04:53<21:24, 142.68s/it]

File /home/fs01/spec1142/Emma/Download_OpenAlex/OA_encoded_works/new_works_encoded_3.tsv uploaded to OpenAlex/works_encoded/new_works_encoded_3.tsv.


 27%|██▋       | 3/11 [07:17<19:04, 143.10s/it]

...

File /home/fs01/spec1142/Emma/Download_OpenAlex/OA_encoded_works/new_works_encoded_10.tsv uploaded to OpenAlex/works_encoded/new_works_encoded_10.tsv.


 91%|█████████ | 10/11 [23:02<02:14, 134.48s/it]

File /home/fs01/spec1142/Emma/Download_OpenAlex/OA_encoded_works/new_works_encoded_11.tsv uploaded to OpenAlex/works_encoded/new_works_encoded_11.tsv.


100%|██████████| 11/11 [24:59<00:00, 136.31s/it]

File /home/fs01/spec1142/Emma/Download_OpenAlex/OA_encoded_works/new_works_encoded_12.tsv uploaded to OpenAlex/works_encoded/new_works_encoded_12.tsv.
